# K-Means Clustering

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Digital-AI-Finance/methods-algorithms/blob/master/notebooks/L03_kmeans.ipynb)

**Learning Objectives:**
- Understand the K-Means algorithm step by step (assign, update, repeat)
- Use the Elbow method and Silhouette analysis to choose K
- Visualize clustering results with Voronoi diagrams
- Apply K-Means to a business problem (customer segmentation with RFM)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, silhouette_samples, pairwise_distances_argmin
from scipy.spatial import Voronoi, voronoi_plot_2d

np.random.seed(42)

plt.rcParams.update({
    'figure.figsize': (10, 6),
    'font.size': 12,
    'axes.titlesize': 14,
    'axes.labelsize': 12,
})

# ML color palette
ML_PURPLE = '#3333B2'
ML_BLUE = '#0066CC'
ML_ORANGE = '#FF7F0E'
ML_GREEN = '#2CA02C'
ML_RED = '#D62728'

print('Setup complete.')

## 1. Generate Data

In [ ]:
# Create 3-cluster blob dataset
X, y_true = make_blobs(n_samples=300, centers=3, cluster_std=1.0, random_state=42)

print(f'Samples: {X.shape[0]}')
print(f'Features: {X.shape[1]}')
print(f'True clusters: {len(np.unique(y_true))}')

In [ ]:
# Visual 1: Raw unlabeled data (single gray color -- we don't know clusters yet!)
fig, ax = plt.subplots(figsize=(10, 6))
ax.scatter(X[:, 0], X[:, 1], c='gray', s=30, alpha=0.6, edgecolors='white')
ax.set_xlabel('Feature 1')
ax.set_ylabel('Feature 2')
ax.set_title('Raw Unlabeled Data -- Can You See the Clusters?')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 2. How K-Means Works

In [ ]:
# Visual 2: 4-panel iteration plot (manual K-Means for pedagogical clarity)
fig, axes = plt.subplots(1, 4, figsize=(20, 4))
rng = np.random.RandomState(42)
centroids = X[rng.choice(len(X), 3, replace=False)]

cluster_colors = [ML_BLUE, ML_ORANGE, ML_GREEN]
titles = ['Init', 'Iteration 1', 'Iteration 2', 'Final (Converged)']

for i, ax in enumerate(axes):
    # Assign points to nearest centroid
    labels = pairwise_distances_argmin(X, centroids)
    colors = [cluster_colors[l] for l in labels]
    ax.scatter(X[:, 0], X[:, 1], c=colors, s=20, alpha=0.6)
    ax.scatter(centroids[:, 0], centroids[:, 1], c=ML_RED, marker='X',
               s=200, edgecolors='black', linewidths=1.5, zorder=5)
    ax.set_title(titles[i], fontsize=12)
    ax.set_xticks([])
    ax.set_yticks([])
    # Update centroids (mean of assigned points)
    centroids = np.array([X[labels == k].mean(axis=0) for k in range(3)])

plt.suptitle('K-Means Algorithm: Step by Step', fontsize=16)
plt.tight_layout()
plt.show()

## 3. Choosing K: Elbow Method

In [ ]:
# Visual 3: Elbow plot (WCSS / inertia vs K=1..10)
inertias = []
k_range = range(1, 11)
for k in k_range:
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    km.fit(X)
    inertias.append(km.inertia_)

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(list(k_range), inertias, 'o-', color=ML_PURPLE, linewidth=2, markersize=8)

# Annotate elbow at K=3
elbow_k = 3
ax.annotate(f'Elbow at K={elbow_k}',
            xy=(elbow_k, inertias[elbow_k - 1]),
            xytext=(elbow_k + 1.5, inertias[elbow_k - 1] + 200),
            fontsize=12, fontweight='bold', color=ML_RED,
            arrowprops=dict(arrowstyle='->', color=ML_RED, lw=2),
            bbox=dict(boxstyle='round', facecolor='lightyellow'))
ax.scatter([elbow_k], [inertias[elbow_k - 1]], c=ML_RED, s=200,
           zorder=5, edgecolors='black')

ax.set_xlabel('K (Number of Clusters)')
ax.set_ylabel('WCSS (Inertia)')
ax.set_title('Elbow Method: Finding the Optimal K')
ax.set_xticks(list(k_range))
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4. Choosing K: Silhouette Analysis

In [ ]:
# Visual 4: Silhouette scores for K=2..8
sil_scores = []
sil_range = range(2, 9)
for k in sil_range:
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = km.fit_predict(X)
    sil_scores.append(silhouette_score(X, labels))

best_sil_k = list(sil_range)[np.argmax(sil_scores)]
best_sil = max(sil_scores)

fig, ax = plt.subplots(figsize=(10, 6))
bars = ax.bar(list(sil_range), sil_scores, color=ML_BLUE, alpha=0.7, edgecolor='white')
# Highlight best
bars[np.argmax(sil_scores)].set_color(ML_GREEN)
bars[np.argmax(sil_scores)].set_alpha(1.0)

ax.annotate(f'Best K={best_sil_k}\nScore={best_sil:.3f}',
            xy=(best_sil_k, best_sil), xytext=(best_sil_k + 1.2, best_sil),
            fontsize=12, fontweight='bold',
            arrowprops=dict(arrowstyle='->', color='black'),
            bbox=dict(boxstyle='round', facecolor='lightyellow'))

ax.set_xlabel('K (Number of Clusters)')
ax.set_ylabel('Average Silhouette Score')
ax.set_title('Silhouette Score vs K')
ax.set_xticks(list(sil_range))
ax.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

print(f'Best K by silhouette: {best_sil_k} (score={best_sil:.3f})')

In [ ]:
# Visual 5: Silhouette plot (knife-shaped bars per cluster)
km_best = KMeans(n_clusters=best_sil_k, random_state=42, n_init=10)
cluster_labels = km_best.fit_predict(X)
sil_vals = silhouette_samples(X, cluster_labels)

fig, ax = plt.subplots(figsize=(10, 6))
y_lower = 10
colors = [ML_BLUE, ML_ORANGE, ML_GREEN, ML_RED, ML_PURPLE]

for i in range(best_sil_k):
    cluster_sil = sil_vals[cluster_labels == i]
    cluster_sil.sort()
    size_cluster = len(cluster_sil)
    y_upper = y_lower + size_cluster
    
    ax.fill_betweenx(np.arange(y_lower, y_upper), 0, cluster_sil,
                     alpha=0.7, color=colors[i % len(colors)])
    ax.text(-0.05, y_lower + 0.5 * size_cluster, f'Cluster {i}', fontsize=11)
    y_lower = y_upper + 10

ax.axvline(x=silhouette_score(X, cluster_labels), color=ML_RED,
           linestyle='--', linewidth=2, label=f'Average ({silhouette_score(X, cluster_labels):.3f})')
ax.set_xlabel('Silhouette Coefficient')
ax.set_ylabel('Cluster')
ax.set_title(f'Silhouette Plot for K={best_sil_k} Clusters')
ax.set_yticks([])
ax.legend(loc='upper right')
ax.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

## 5. Final Clustering Result

In [ ]:
# Visual 6: Scatter with colored clusters + centroid markers + Voronoi overlay
fig, ax = plt.subplots(figsize=(10, 6))

colors_map = [ML_BLUE, ML_ORANGE, ML_GREEN]
for i in range(best_sil_k):
    mask = cluster_labels == i
    ax.scatter(X[mask, 0], X[mask, 1], c=colors_map[i % len(colors_map)],
               s=40, alpha=0.7, edgecolors='white', label=f'Cluster {i}')

# Centroids
centers = km_best.cluster_centers_
ax.scatter(centers[:, 0], centers[:, 1], c=ML_RED, marker='X',
           s=300, edgecolors='black', linewidths=2, zorder=10, label='Centroids')

# Voronoi overlay
vor = Voronoi(centers)
voronoi_plot_2d(vor, ax=ax, show_vertices=False, show_points=False,
                line_colors='black', line_width=2, line_alpha=0.5)

ax.set_xlabel('Feature 1')
ax.set_ylabel('Feature 2')
ax.set_title(f'Final K-Means Clustering (K={best_sil_k}) with Voronoi Regions')
ax.legend(loc='upper right')
ax.grid(True, alpha=0.3)

# Set axis limits to data range
ax.set_xlim(X[:, 0].min() - 1, X[:, 0].max() + 1)
ax.set_ylim(X[:, 1].min() - 1, X[:, 1].max() + 1)

plt.tight_layout()
plt.show()

## 6. Customer Segmentation Example (RFM)

In [ ]:
# Generate simple RFM data: 4 customer segments
np.random.seed(42)
n_per_segment = 75

# Segment 0: High-value loyal (low recency, high frequency, high monetary)
seg0 = np.column_stack([
    np.random.normal(10, 5, n_per_segment),    # Recency (days)
    np.random.normal(25, 5, n_per_segment),    # Frequency
    np.random.normal(500, 100, n_per_segment)  # Monetary
])

# Segment 1: At-risk (high recency, medium frequency, medium monetary)
seg1 = np.column_stack([
    np.random.normal(90, 15, n_per_segment),
    np.random.normal(12, 4, n_per_segment),
    np.random.normal(250, 80, n_per_segment)
])

# Segment 2: New customers (medium recency, low frequency, low monetary)
seg2 = np.column_stack([
    np.random.normal(30, 10, n_per_segment),
    np.random.normal(3, 2, n_per_segment),
    np.random.normal(80, 30, n_per_segment)
])

# Segment 3: Churned (very high recency, low frequency, low monetary)
seg3 = np.column_stack([
    np.random.normal(180, 30, n_per_segment),
    np.random.normal(2, 1, n_per_segment),
    np.random.normal(50, 20, n_per_segment)
])

rfm_data = np.vstack([seg0, seg1, seg2, seg3])
rfm_data = np.maximum(rfm_data, 0)  # No negative values

rfm_df = pd.DataFrame(rfm_data, columns=['Recency', 'Frequency', 'Monetary'])
print(rfm_df.describe().round(1))

# Scale before clustering
scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm_data)

# Cluster
km_rfm = KMeans(n_clusters=4, random_state=42, n_init=10)
rfm_df['Cluster'] = km_rfm.fit_predict(rfm_scaled)

print(f'\nCluster sizes: {rfm_df["Cluster"].value_counts().sort_index().to_dict()}')

In [ ]:
# Visual 7: 2D scatter of RFM clusters (Recency vs Monetary)
fig, ax = plt.subplots(figsize=(10, 6))

cluster_colors = [ML_BLUE, ML_ORANGE, ML_GREEN, ML_PURPLE]
cluster_names = ['Loyal High-Value', 'At-Risk', 'New Customers', 'Churned']

for i in range(4):
    mask = rfm_df['Cluster'] == i
    ax.scatter(rfm_df.loc[mask, 'Recency'], rfm_df.loc[mask, 'Monetary'],
               c=cluster_colors[i], s=40, alpha=0.7, edgecolors='white',
               label=f'Cluster {i}')

# Centroid markers in original space
centers_orig = scaler.inverse_transform(km_rfm.cluster_centers_)
ax.scatter(centers_orig[:, 0], centers_orig[:, 2], c=ML_RED, marker='X',
           s=300, edgecolors='black', linewidths=2, zorder=10, label='Centroids')

ax.set_xlabel('Recency (days since last purchase)')
ax.set_ylabel('Monetary (total spend)')
ax.set_title('Customer Segmentation: Recency vs Monetary')
ax.legend(loc='upper right')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Visual 8: Cluster profile grouped bar chart (mean R, F, M per cluster)
cluster_means = rfm_df.groupby('Cluster')[['Recency', 'Frequency', 'Monetary']].mean()

fig, ax = plt.subplots(figsize=(10, 6))
x = np.arange(len(cluster_means))
width = 0.25

# Normalize each metric to [0, 1] for comparable bar heights
means_norm = cluster_means.copy()
for col in means_norm.columns:
    means_norm[col] = means_norm[col] / means_norm[col].max()

bars_r = ax.bar(x - width, means_norm['Recency'], width, color=ML_BLUE, label='Recency', alpha=0.85)
bars_f = ax.bar(x, means_norm['Frequency'], width, color=ML_ORANGE, label='Frequency', alpha=0.85)
bars_m = ax.bar(x + width, means_norm['Monetary'], width, color=ML_GREEN, label='Monetary', alpha=0.85)

# Add actual mean values on top of bars
for bars, col in zip([bars_r, bars_f, bars_m], ['Recency', 'Frequency', 'Monetary']):
    for bar, val in zip(bars, cluster_means[col]):
        ax.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.02,
                f'{val:.0f}', ha='center', va='bottom', fontsize=9)

ax.set_xlabel('Cluster')
ax.set_ylabel('Normalized Mean Value')
ax.set_title('Cluster Profiles: Average RFM Values')
ax.set_xticks(x)
ax.set_xticklabels([f'Cluster {i}' for i in range(4)])
ax.legend(loc='upper right')
ax.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

## Summary

**Key Takeaways:**
- **K-Means iterates** assign-to-nearest-centroid and update-centroid until convergence -- simple but powerful
- **The Elbow Method** looks for where adding more clusters gives diminishing returns (WCSS drop slows down)
- **Silhouette Analysis** measures how well each point fits its cluster vs the next closest cluster (higher is better)
- **Always scale features** before clustering -- K-Means uses Euclidean distance, so unscaled features dominate